In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
from numpy import array as arr
import time
sys.path.append('../src')
import utils

# sklearn models
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree

import torch
from torch import nn

cred = (234/255, 51/255, 86/255)
cblue = (57/255, 138/255, 242/255)
out_dir = '../results/sim_linear'
os.makedirs(out_dir, exist_ok=True)

# rulefit
- must be installed with: `pip install git+git://github.com/christophM/rulefit.git`
- some docs: `https://github.com/christophM/rulefit`
- original paper: `http://statweb.stanford.edu/~jhf/ftp/RuleFit.pdf`

In [ ]:
import numpy as np
import pandas as pd

from rulefit import RuleFit

boston_data = pd.read_csv("boston.csv", index_col=0)

y = boston_data.medv.values
X = boston_data.drop("medv", axis=1)
features = X.columns
X = X.values

rf = RuleFit()
rf.fit(X, y, feature_names=features)

preds = rf.predict(X)
print(f'train mse: {np.mean(np.square(preds-y)):0.2f}')

*inspect the rules*
- rule is how the feature is constructed
- coef is its weight in the final linear model
- support is how many points it applies to

In [ ]:
rules = rf.get_rules()

rules = rules[rules.coef != 0].sort_values("support", ascending=False)

print(rules[['rule', 'coef', 'support']]) #.sort_values('coef'))

# scalable bayesian rule lists
- docs at `https://github.com/myaooo/pysbrl`
- requires installing some `c` packages (ex. instructions [here](https://coral.ise.lehigh.edu/jild13/2016/07/11/hello/))

# optimal classification tree
- docs here: `https://github.com/pan5431333/pyoptree`
- note: this implementation is still unstable

In [ ]:
import pandas as pd
from pyoptree.optree import OptimalHyperTreeModel, OptimalTreeModel

data = pd.DataFrame({
        "index": ['A', 'C', 'D', 'E', 'F'],
        "x1": [1, 2, 2, 2, 3],
        "x2": [1, 2, 1, 0, 1],
        "y": [1, 1, -1, -1, -1]
    })
test_data = pd.DataFrame({
    "index": ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    "x1": [1, 1, 2, 2, 2, 3, 3],
    "x2": [1, 2, 2, 1, 0, 1, 0],
    "y": [1, 1, 1, -1, -1, -1, -1]
})
model = OptimalHyperTreeModel(["x1", "x2"], "y", tree_depth=2, N_min=1, alpha=0.1, solver_name="cplex")
model.train(data)

print(model.predict(test_data))